# E-Commerce Order Dataset - Comprehensive Data Cleaning & Validation

This notebook performs extensive data quality analysis and cleaning on an order dataset, identifying and resolving various data inconsistencies and business rule violations.

## 1. Load and Explore Dataset

**Question:** What is the structure and quality of the dataset? How many records are there, and what are the basic statistics?

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
from datetime import datetime

# Load dataset
df = pd.read_csv('Dataset_1.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\n" + "="*80)
print("Data Types:")
print(df.dtypes)
print("\n" + "="*80)
print("First Few Rows:")
print(df.head(10))
print("\n" + "="*80)
print("Missing Values Count:")
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Missing_Percentage': missing_percentage.values
})
print(missing_df)
print("\n" + "="*80)
print("Statistical Summary:")
print(df.describe())

## 2. Identify Non-Numeric Order Amounts with Delivered Status

**Question:** Which rows have non-numeric order_amount values (like ₹ symbols or 'N/A') but are still marked as delivered? Are these data quality issues or valid business scenarios?

In [ ]:
def is_numeric(val):
    """Check if a value is numeric or can be converted to numeric"""
    if pd.isna(val):
        return False
    try:
        float(val)
        return True
    except:
        return False

def is_delivered_status(val):
    """Check if value indicates delivered status"""
    if pd.isna(val):
        return False
    val_str = str(val).lower().strip()
    return val_str in ['true', 'yes', '1', 'delivered']

# Find non-numeric amounts with delivered status
non_numeric_mask = ~df['order_amount'].apply(is_numeric)
delivered_mask = df['is_delivered'].apply(is_delivered_status)

non_numeric_delivered = df[non_numeric_mask & delivered_mask].copy()

print("ANALYSIS: Non-Numeric Order Amounts with Delivered Status")
print("="*80)
print(f"Total records with non-numeric amounts AND delivered status: {len(non_numeric_delivered)}")
print(f"Percentage of total records: {(len(non_numeric_delivered)/len(df))*100:.2f}%")

if len(non_numeric_delivered) > 0:
    print("\nSample Records:")
    print(non_numeric_delivered[['order_id', 'customer_id', 'order_amount', 'is_delivered']].head(15))
    print("\nUnique non-numeric values found:")
    print(non_numeric_delivered['order_amount'].value_counts())
else:
    print("\nNo records found with this combination.")

## 3. Convert and Validate Order Dates

**Question:** Can we successfully convert all order_date values to datetime format? Which orders have negative or illogical delivery_days after date conversion?

In [ ]:
def parse_date(date_str):
    """Parse date with multiple formats"""
    if pd.isna(date_str) or date_str == '':
        return None
    
    date_str = str(date_str).strip()
    
    # Formats to try: YYYY-MM-DD, YYYY/MM/DD, DD/MM/YYYY
    formats = ['%Y-%m-%d', '%Y/%m/%d', '%d/%m/%Y']
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    return None

# Create a working copy with converted dates
df_working = df.copy()
df_working['order_date_converted'] = df['order_date'].apply(parse_date)

print("DATE CONVERSION ANALYSIS")
print("="*80)
print(f"Total records: {len(df)}")
print(f"Successfully converted dates: {df_working['order_date_converted'].notna().sum()}")
print(f"Failed to convert: {df_working['order_date_converted'].isna().sum()}")

# Find records with negative delivery days (after valid date conversion)
df_working['delivery_days_numeric'] = pd.to_numeric(df['delivery_days'], errors='coerce')

negative_delivery = df_working[(df_working['order_date_converted'].notna()) & 
                               (df_working['delivery_days_numeric'] < 0)].copy()

print(f"\nRecords with NEGATIVE delivery_days (after valid date conversion):")
print(f"Count: {len(negative_delivery)}")
print(f"Percentage: {(len(negative_delivery)/len(df))*100:.2f}%")

if len(negative_delivery) > 0:
    print("\nSample Records with Negative Delivery Days:")
    print(negative_delivery[['order_id', 'order_date', 'delivery_days', 'order_date_converted']].head(15))
    print("\nDistribution of Negative Delivery Days:")
    print(negative_delivery['delivery_days_numeric'].value_counts().sort_index())
else:
    print("No records found with this issue.")

## 4. Standardize Payment Modes and Calculate Distribution

**Question:** How many different payment mode variations exist? What is the percentage share of each standardized payment mode per city?

In [ ]:
def standardize_payment_mode(mode):
    """Standardize payment mode values"""
    if pd.isna(mode) or mode == '':
        return 'Unknown'
    
    mode_str = str(mode).lower().strip()
    
    if 'debit' in mode_str:
        return 'Debit Card'
    elif 'credit' in mode_str:
        return 'Credit Card'
    elif 'upi' in mode_str:
        return 'UPI'
    elif 'cash' in mode_str:
        return 'Cash'
    elif 'card' in mode_str:
        return 'Card'
    else:
        return 'Unknown'

# Create standardized payment mode
df_working['payment_mode_std'] = df['payment_mode'].apply(standardize_payment_mode)

print("PAYMENT MODE STANDARDIZATION")
print("="*80)
print("Original Payment Modes:")
print(df['payment_mode'].value_counts(dropna=False))

print("\n" + "="*80)
print("Standardized Payment Modes:")
print(df_working['payment_mode_std'].value_counts(dropna=False))

print("\n" + "="*80)
print("Percentage Distribution of Payment Modes Per City:")
print("="*80)

for city in sorted(df_working[df_working['city'].notna()]['city'].unique()):
    city_data = df_working[df_working['city'] == city]
    payment_dist = (city_data['payment_mode_std'].value_counts() / len(city_data) * 100).round(2)
    print(f"\n{city} (Total Orders: {len(city_data)}):")
    print(payment_dist.to_string())
    print("-" * 40)

## 5. Detect Invalid Customer Ages with High Order Amounts

**Question:** Which customers have invalid ages (negative, missing, or non-numeric) but place orders with amounts above the dataset median? Are these data entry errors or fraudulent activities?

In [ ]:
def is_valid_age(age):
    """Check if age is valid (numeric and between 0-120)"""
    try:
        age_val = float(age)
        return 0 <= age_val <= 120
    except:
        return False

def clean_order_amount(amount):
    """Clean order amount by removing currency symbols"""
    if pd.isna(amount) or amount == '':
        return np.nan
    
    amount_str = str(amount).strip()
    # Remove ₹ symbol and any other non-numeric characters except decimal point
    amount_str = amount_str.replace('₹', '').strip()
    
    try:
        return float(amount_str)
    except:
        return np.nan

# Clean order amounts
df_working['order_amount_numeric'] = df['order_amount'].apply(clean_order_amount)

# Calculate dataset median
median_amount = df_working['order_amount_numeric'].median()
print("ORDER AMOUNT STATISTICS")
print("="*80)
print(f"Dataset Median Order Amount: ₹{median_amount:.2f}")
print(f"Valid order amounts: {df_working['order_amount_numeric'].notna().sum()}")
print(f"Invalid order amounts: {df_working['order_amount_numeric'].isna().sum()}")

# Find customers with invalid age but high order amount
invalid_age_mask = ~df_working['customer_age'].apply(is_valid_age)
high_amount_mask = df_working['order_amount_numeric'] > median_amount

invalid_age_high_amount = df_working[invalid_age_mask & high_amount_mask].copy()

print("\n" + "="*80)
print("INVALID AGES WITH HIGH ORDER AMOUNTS")
print("="*80)
print(f"Total records: {len(invalid_age_high_amount)}")
print(f"Percentage of total: {(len(invalid_age_high_amount)/len(df))*100:.2f}%")

if len(invalid_age_high_amount) > 0:
    print(f"\nSample Records:")
    print(invalid_age_high_amount[['order_id', 'customer_id', 'customer_age', 'order_amount_numeric']].head(15))
    
    print(f"\nInvalid Age Values Found:")
    print(invalid_age_high_amount['customer_age'].value_counts(dropna=False))
else:
    print("No records found with this combination.")

## 6. Find Duplicate Customer IDs Across Multiple Cities

**Question:** Which customers (by customer_id) have placed orders in multiple cities? What are their transaction patterns across locations?

In [ ]:
print("DUPLICATE CUSTOMER IDs ACROSS MULTIPLE CITIES")
print("="*80)

# Filter valid customer IDs (non-empty, non-null)
df_valid_cust = df_working[(df_working['customer_id'].notna()) & (df_working['customer_id'] != '')].copy()

# Find customers with orders in multiple cities
customer_cities = df_valid_cust[df_valid_cust['city'].notna()].groupby('customer_id')['city'].apply(lambda x: x.unique())
multi_city_customers = customer_cities[customer_cities.apply(len) > 1]

print(f"\nTotal unique customers: {df_valid_cust['customer_id'].nunique()}")
print(f"Customers with valid city information: {df_valid_cust[df_valid_cust['city'].notna()]['customer_id'].nunique()}")
print(f"Customers ordering from multiple cities: {len(multi_city_customers)}")
print(f"Percentage: {(len(multi_city_customers)/df_valid_cust['customer_id'].nunique()*100):.2f}%")

if len(multi_city_customers) > 0:
    print("\n" + "="*80)
    print("Details of Multi-City Customers:")
    print("="*80)
    
    for customer_id in multi_city_customers.index[:10]:  # Show first 10
        cities = multi_city_customers[customer_id]
        customer_orders = df_valid_cust[df_valid_cust['customer_id'] == customer_id]
        city_counts = customer_orders['city'].value_counts()
        
        print(f"\nCustomer: {customer_id}")
        print(f"  Cities Ordered From: {', '.join(cities)}")
        print(f"  Total Orders: {len(customer_orders)}")
        print(f"  Orders per City:")
        for city, count in city_counts.items():
            print(f"    - {city}: {count}")
else:
    print("\nNo customers with multi-city orders found.")

## 7. Flag Missing Delivery Days with Delivered Status

**Question:** How many orders are marked as delivered but have missing delivery_days values? Is this a data quality issue?

In [ ]:
print("MISSING DELIVERY DAYS WITH DELIVERED STATUS")
print("="*80)

# Create flag for missing delivery days with delivered status
missing_delivery_days = (df_working['delivery_days'].isna()) | (df_working['delivery_days'] == '')
df_working['flag_missing_delivery_delivered'] = missing_delivery_days & df_working['is_delivered'].apply(is_delivered_status)

flagged_records = df_working[df_working['flag_missing_delivery_delivered']].copy()

print(f"Total records with missing delivery_days: {missing_delivery_days.sum()}")
print(f"Records with missing delivery_days AND delivered status: {len(flagged_records)}")
print(f"Percentage of total: {(len(flagged_records)/len(df))*100:.2f}%")

if len(flagged_records) > 0:
    print("\nSample Flagged Records:")
    print(flagged_records[['order_id', 'delivery_days', 'is_delivered', 'order_date']].head(15))
    
    print("\nBreakdown by is_delivered value:")
    print(flagged_records['is_delivered'].value_counts(dropna=False))
else:
    print("\nNo records found with this issue.")

## 8. Calculate Average Order Value by Payment Mode

**Question:** What is the average order value for each standardized payment mode after removing invalid amounts?

In [ ]:
print("AVERAGE ORDER VALUE BY PAYMENT MODE")
print("="*80)

# Calculate average order value by payment mode (excluding invalid amounts)
valid_data = df_working[df_working['order_amount_numeric'].notna()].copy()

avg_by_payment = valid_data.groupby('payment_mode_std')['order_amount_numeric'].agg([
    ('Count', 'count'),
    ('Average', 'mean'),
    ('Median', 'median'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('Std Dev', 'std')
]).round(2)

print("\nAverage Order Value by Payment Mode (Cleaned Data):")
print(avg_by_payment)

print("\n" + "="*80)
print("Key Insights:")
print("="*80)

for payment_mode in avg_by_payment.index:
    if avg_by_payment.loc[payment_mode, 'Count'] > 0:
        avg_val = avg_by_payment.loc[payment_mode, 'Average']
        count = avg_by_payment.loc[payment_mode, 'Count']
        print(f"{payment_mode:15} - Avg: ₹{avg_val:8.2f} | Count: {int(count):4}")


## 9. Identify Cities with Missing Order Dates

**Question:** Which cities have more than 30% of their orders with missing order_date values? What is the data quality distribution?

In [ ]:
print("CITIES WITH MISSING ORDER DATES")
print("="*80)

# Analyze missing order dates by city
city_data = df_working[df_working['city'].notna()].copy()

missing_dates_by_city = city_data.groupby('city').agg({
    'order_date': lambda x: (x.isna().sum()),
    'order_id': 'count'
}).rename(columns={'order_date': 'Missing_Count', 'order_id': 'Total_Count'})

missing_dates_by_city['Missing_Percentage'] = (
    (missing_dates_by_city['Missing_Count'] / missing_dates_by_city['Total_Count']) * 100
).round(2)

missing_dates_by_city = missing_dates_by_city.sort_values('Missing_Percentage', ascending=False)

print(missing_dates_by_city)

# Find cities with >30% missing dates
high_missing = missing_dates_by_city[missing_dates_by_city['Missing_Percentage'] > 30]

print("\n" + "="*80)
print(f"Cities with >30% Missing Order Dates: {len(high_missing)}")
print("="*80)

if len(high_missing) > 0:
    for city in high_missing.index:
        pct = high_missing.loc[city, 'Missing_Percentage']
        count = high_missing.loc[city, 'Missing_Count']
        total = high_missing.loc[city, 'Total_Count']
        print(f"{city:15} - Missing: {int(count):4}/{int(total):4} ({pct:6.2f}%)")
else:
    print("No cities found with >30% missing order dates.")

## 10. Normalize Order Amount and Detect Outliers

**Question:** Within each city, which orders have normalized amounts exceeding 2 standard deviations from the mean? These represent outlier transactions.

In [ ]:
from scipy import stats

print("NORMALIZE ORDER AMOUNT AND DETECT OUTLIERS")
print("="*80)

# Normalize within each city
df_working['normalized_amount'] = np.nan
df_working['is_outlier_2std'] = False

outliers_list = []

for city in df_working[df_working['city'].notna()]['city'].unique():
    city_mask = (df_working['city'] == city) & (df_working['order_amount_numeric'].notna())
    city_data = df_working[city_mask].copy()
    
    if len(city_data) > 1:
        mean = city_data['order_amount_numeric'].mean()
        std = city_data['order_amount_numeric'].std()
        
        # Normalize
        df_working.loc[city_mask, 'normalized_amount'] = (
            (city_data['order_amount_numeric'] - mean) / std if std > 0 else 0
        )
        
        # Find outliers (>2 or <-2 standard deviations)
        outlier_mask = (abs(df_working.loc[city_mask, 'normalized_amount']) > 2)
        df_working.loc[city_mask[outlier_mask], 'is_outlier_2std'] = True
        
        outlier_count = outlier_mask.sum()
        
        if outlier_count > 0:
            outliers_list.append({
                'city': city,
                'outlier_count': outlier_count,
                'total_orders': len(city_data),
                'percentage': (outlier_count / len(city_data) * 100)
            })

print("Outliers Detected (>2 Standard Deviations) by City:")
print("="*80)

if outliers_list:
    outliers_df = pd.DataFrame(outliers_list).sort_values('percentage', ascending=False)
    for idx, row in outliers_df.iterrows():
        print(f"{row['city']:15} - Outliers: {row['outlier_count']:3}/{row['total_orders']:3} ({row['percentage']:6.2f}%)")
else:
    print("No outliers found.")

print("\n" + "="*80)
print("Sample Outlier Records:")
print("="*80)

outlier_records = df_working[df_working['is_outlier_2std']].sort_values('normalized_amount', key=abs, ascending=False)
if len(outlier_records) > 0:
    print(outlier_records[['order_id', 'city', 'order_amount_numeric', 'normalized_amount']].head(15))
else:
    print("No outlier records found.")

## 11. Validate Boolean Values in is_delivered

**Question:** Are there inconsistencies between the is_delivered status and delivery_days? Can we standardize the boolean-like values?